In [39]:

import pandas as pd
import numpy as np
from datetime import datetime
import json



In [40]:
dataEvents = pd.read_csv('E:/Documentos/PCEO/5/Informatica/TFG/datos/anonamyze_all_data_collection_v2.csv', sep=";")

In [41]:
typeMapping = ['Sandbox~SAND', '1. One Box~Tutorial', '2. Separated Boxes~Tutorial', '3. Rotate a Pyramid~Tutorial', '4. Match Silhouettes~Tutorial', '5. Removing Objects~Tutorial', '6. Stretch a Ramp~Tutorial', '7. Max 2 Boxes~Tutorial', '8. Combine 2 Ramps~Tutorial', '9. Scaling Round Objects~Tutorial',
               'Square Cross-Sections~Easy Puzzles', 'Bird Fez~Easy Puzzles', 'Pi Henge~Easy Puzzles', '45-Degree Rotations~Easy Puzzles',  'Pyramids are Strange~Easy Puzzles', 'Boxes Obscure Spheres~Easy Puzzles', 'Object Limits~Easy Puzzles', 'Not Bird~Easy Puzzles', 'Angled Silhouette~Easy Puzzles',
               'Warm Up~Hard Puzzles','Tetromino~Hard Puzzles', 'Stranger Shapes~Hard Puzzles', 'Sugar Cones~Hard Puzzles', 'Tall and Small~Hard Puzzles', 'Ramp Up and Can It~Hard Puzzles', 'More Than Meets Your Eye~Hard Puzzles', 'Unnecessary~Hard Puzzles', 'Zzz~Hard Puzzles', 'Bull Market~Hard Puzzles', 'Few Clues~Hard Puzzles', 'Orange Dance~Hard Puzzles', 'Bear Market~Hard Puzzles']

tutorialPuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Tutorial'):
        tutorialPuzzles.append(desc[0])

advancedPuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Hard Puzzles'):
        advancedPuzzles.append(desc[0])


intermediatePuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Easy Puzzles'):
        intermediatePuzzles.append(desc[0])

allPuzzles = []
for puzzle in typeMapping:
    desc = puzzle.split("~")
    allPuzzles.append(desc[0])



def computeUserHistorial(dataEvents, group = 'all', user = 'all',timeLimit = pd.to_datetime('2022-09-10 13:40:17.975299-04:00',utc=True) ):

    dataEvents['time'] = pd.to_datetime(dataEvents['time'],utc=True)
    dataEvents = dataEvents.sort_values('time')
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]

    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']

    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    # filtering to only take the user passed as argument
    if(user != 'all'):
        dataEvents = dataEvents[dataEvents['user'].isin(user)]
    # filtering to only take the events before the timeLimit
    dataEvents = dataEvents.loc[dataEvents['time'] < timeLimit]


    completed = dict()
    tutorialCompleted = dict()
    intermediateCompleted = dict()
    advancedCompleted = dict()
    users = []
    userAttempts = dict()
    userWins = dict()

    for user in dataEvents['group_user_id'].unique():

        users.append(user)
        tutorialCompleted[user] = 0
        intermediateCompleted[user] = 0
        advancedCompleted[user] = 0
        userAttempts[user] = 0
        userWins[user] = 0

        user_events = dataEvents[dataEvents['group_user_id'] == user]
        user_puzzle_key = None

        for enum, event in user_events.iterrows():

            if(event['type'] in ['ws-start_level', 'ws-puzzle_started']):
                user_puzzle_key = event['group'] + '~' + event['user'] + '~' + json.loads(event['data'])['task_id']
                if(user_puzzle_key not in completed.keys()): completed[user_puzzle_key] = 0

            if(event['type'] == 'ws-puzzle_started'):
                userAttempts[user] +=1
            elif(event['type'] == 'ws-puzzle_complete'):
                userWins[user] += 1
                if completed[user_puzzle_key] == 0 :
                    llave = user_puzzle_key.split("~")
                    puzzle = llave[2]
                    if (puzzle in tutorialPuzzles):
                        tutorialCompleted[user] += 1
                    elif (puzzle in intermediatePuzzles):
                        intermediateCompleted[user] += 1
                    elif (puzzle in advancedPuzzles):
                        advancedCompleted[user] += 1
                completed[user_puzzle_key] = 1

    userHistorialList = []

    for key in users :

        key_split = key.split('~')

        if userAttempts[key] == 0 or userWins[key] == 0:
            attempts_per_puzzle = 0
        else:
            attempts_per_puzzle = round((userAttempts[key]/userWins[key]),2)

        userHistorialList.append([key_split[0], key_split[1], round((tutorialCompleted[key]/9)*100,2),round((intermediateCompleted[key]/9)*100,2),round((advancedCompleted[key]/13)*100,2),attempts_per_puzzle ])

    userHistorial = pd.DataFrame(userHistorialList, columns=['group','user','percentage_tutorial','percentage_intermediate','percentage_advanced','attempts_per_puzzle'])

    return userHistorial

In [42]:
userHistorial = computeUserHistorial(dataEvents)
userHistorialAll = computeUserHistorial(dataEvents)

0
1.7
1.7
2.67
2.67
1.76
1.76
2.16
2.16
1.62
1.62
1.73
1.73
2.88
2.88
1.38
1.38
1.56
1.56
2.25
2.25
2.08
2.08
4.0
4.0
1.64
1.64
2.29
2.29
1.46
1.46
1.67
1.67
3.0
3.0
3.67
3.67
1.4
1.4
1.33
1.33
1.75
1.75
1.67
1.67
1.9
1.9
1.78
1.78
1.1
1.1
2.0
2.0
0
2.27
2.27
0
0
4.0
4.0
1.6
1.6
1.18
1.18
1.57
1.57
1.62
1.62
1.43
1.43
2.46
2.46
1.64
1.64
1.68
1.68
3.0
3.0
1.67
1.67
2.6
2.6
2.17
2.17
2.6
2.6
1.44
1.44
2.31
2.31
3.08
3.08
2.4
2.4
1.39
1.39
1.6
1.6
1.17
1.17
1.36
1.36
1.13
1.13
0
1.3
1.3
1.25
1.25
0
0
2.44
2.44
1.77
1.77
2.67
2.67
1.95
1.95
0
1.62
1.62
1.45
1.45
2.5
2.5
2.19
2.19
1.84
1.84
1.33
1.33
1.14
1.14
1.5
1.5
2.21
2.21
1.91
1.91
1.53
1.53
1.13
1.13
0
2.55
2.55
1.78
1.78
2.61
2.61
1.55
1.55
2.29
2.29
1.29
1.29
1.61
1.61
1.28
1.28
1.48
1.48
1.35
1.35
1.89
1.89
1.5
1.5
3.67
3.67
3.33
3.33
2.0
2.0
0
1.35
1.35
2.67
2.67
0
2.0
2.0
1.5
1.5
2.31
2.31
1.5
1.5
0
0
0
6.0
6.0
8.0
8.0
4.0
4.0
4.0
4.0
3.0
3.0
4.0
4.0
3.0
3.0
1.71
1.71
2.0
2.0
3.17
3.17
1.0
1.0
1.29
1.29
1.36
1.36
0
3.5
3.5
2.67

In [43]:
userHistorial.to_csv("userHistorialOutput.csv", decimal = ".",sep =";", mode='w')
userHistorialAll.to_csv("userHistorialAllOutput.csv", decimal = ".",sep =";", mode='w')